## Reporte: Análisis de respuestas para cuestionario "Español neutro"

El siguiente reporte tiene como objetivo mostrar los resultados obtenidos de la encuesta, mostrando datos agregados y gráficos que sirvan para su rápida interpretación.

### Preparación del entorno de trabajo

In [120]:
import re
import pickle
import os.path
import pandas as pd
import matplotlib as plot

from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

#### Preparar credenciales de google
Función auxiliar que lee el archivo de credenciales y las refresca si es necesario.

In [121]:
def gsheet_api_check(scopes):
    creds = None
    if os.path.exists('../conf/token.pickle'):
        with open('../conf/token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                '../conf/token.json', scopes)
            creds = flow.run_local_server(port=0)
        with open('../conf/token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

#### Obtener los datos de gsheet
Función auxiliar que permite obtener los datos crudos de gsheet a través del uso de credenciales.

In [122]:
def pull_sheet_data(scopes, spreadsheet_id, data_to_pull):
    creds = gsheet_api_check(scopes)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=spreadsheet_id,
        range=data_to_pull).execute()
    values = result.get('values', [])

    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=spreadsheet_id,
                                  range=data_to_pull).execute()
        data = rows.get('values')
        return data

#### Obtener los 3 resultados más altos
Función auxiliar para obtener los 3 resultados más altos.

In [123]:
def get_top_3_answers_by_country(dataframe, answer_col):
    return dataframe.groupby(['country', answer_col])[answer_col].count()\
    .reset_index(name='count')\
    .sort_values(by=['country', 'count'], ascending=[True, False])\
    .groupby('country')\
    .head(3)\
    .style\
    .hide_index()

#### Imputaciones generales a los datos
Función auxiliar que permite realizar limpieza de datos genérica a las columnas pasadas como parámetro.

In [124]:
def generic_data_cleansing(col_dataframe):
    return col_dataframe.map(lambda x: '' if x is None else x).map(lambda x: x.capitalize()).map(lambda x: x.strip())

#### Leer los datos de la encuesta
En el siguiente fragmento de código se realiza la lectura del gsheet, para convertirlo en DataFrame de pandas.

In [125]:
scopes = ['https://www.googleapis.com/auth/spreadsheets.readonly']
sheet_id = "1WaXRSwWmr0tQ0iTTj4FV__hM1dtoXvi-89sIV-4GeAU"
data_to_pull = "Respuestas de formulario 1"

data = pull_sheet_data(scopes, sheet_id, data_to_pull)
col_names = [f'{name}_{i}' for i, name in enumerate(data[0])]

df = pd.DataFrame(data[1:], columns=col_names)

rename_columns = {
    'Marca temporal_0': 'timestamp',
    'País de origen_1': 'country',
    'Edad_2': 'age'
}

for i, name in enumerate(df.columns[3:]):
    rename_columns[name] = f'answer_q{i}'

df = df.rename(columns=rename_columns)

df.head()

,timestamp,country,age,answer_q0,answer_q1,answer_q2,answer_q3,answer_q4,answer_q5,answer_q6,...,answer_q34,answer_q35,answer_q36,answer_q37,answer_q38,answer_q39,answer_q40,answer_q41,answer_q42,answer_q43
0,28/6/2021 17:10:21,arg,32,reposera,delantal,pulsera,anillo,remera,diario,cava,...,choclo,chupetin,ananao piña,calabaza anco,calabaza redonda,leche condesnada,chicle,inpalpable,resaca,cheto'? nose si es tan despectivo como eso
1,28/6/2021 16:11:53,Argentina,23,reposera,guardapolvo,pulcera,anillo,remera,diario,cava,...,choclo,chupetín,ananá,calabaza,zapallo,leche condensada,chicle,azúcar impalpable,resaca,cheto/a
2,28/6/2021 16:22:29,Argentina,24,Reposera,Guardapolvo,Pulsera,Anillo,Remera,Diario,Bodega,...,Choclo,Chupetín,Ananá,Calabaza,Zapallo,Leche condensada,Chicle,Impalpable,Resaca,cheta
3,28/6/2021 16:30:05,Argentina,23,reposera,delantal,pulsera,anillo,remera,diarios,cava,...,choclo,chupetin,ananá,calabaza,zapallo,leche condensada,chicle,azucar impalpable,resaca,cheto
4,28/6/2021 16:30:27,Argentina,23,Reposera,Guardapolvo,Pulsera,Anillo,Remera,Diario,Bodega,...,Choclo,Chupetin,Anana,Zapallo anco,Zapallo cabutia,Leche condensada,Chicle,Impalpable,Resaca,Cheta


### Limpieza de datos
A continuación se realizará una limpieza de los datos para cada una de las columnas dentro del dataframe. Primero se ejecutará una limpieza general y luego una particular para cada caso.

#### Columna: country

In [126]:
# Imputaciones generales
df['country'] = df['country']\
    .map(lambda x: x.capitalize())\
    .map(lambda x: x.strip())

# Imputaciones particulares
df['country'] = df['country']\
    .map(lambda x: re.sub('^Arg$|^Argentina \(santa cruz\)', 'Argentina', x))\
    .map(lambda x: re.sub('^España \( país vasco\)|País vasco', 'España', x))\
    .map(lambda x: re.sub('Mexico', 'México', x))\
    .map(lambda x: re.sub('Peru', 'Perú', x))

df.groupby(['country'])['country'].count()

country
Argentina        214
Bolivia            1
Brasil             1
Chile             43
Colombia          16
Costa rica         1
Cuba               1
Ecuador            8
El salvador        1
España            45
Guatemala          2
México            38
Paraguay           6
Perú              29
Puerto rico        2
Reino unido        1
Rumanía            1
United states      1
Uruguay           13
Venezuela         21
Name: country, dtype: int64

#### Columna: age

In [127]:
# Imputaciones generales
df['age'] = df['age'].map(lambda x: x.split(' ')[0])

df.groupby(['age'])['age'].count()

age
15     2
16     3
17    10
18     3
19     8
20     8
21    15
22    24
23    55
24    33
25    29
26    19
27    11
28    14
29    16
30    18
31    12
32    13
33    10
34    12
35     7
36     9
37     8
38    10
39     7
40     8
41     6
42    10
43     5
44     1
45     4
46     9
47     3
48     5
49     3
50     5
51     4
52     2
53     3
54     3
55     1
56     3
57     2
58     2
59     1
61     2
62     1
63     1
64     3
66     1
81     1
Name: age, dtype: int64

### Limpieza y análisis de datos

#### Pregunta: ¿Cómo le dicen a este elemento en tu país? (answer_q0)
Imagen: <img src='https://lh4.googleusercontent.com/uN_Ny5XZzBTqCOGTwu3oU0HY37DoSG85cJIrDh430_VA7UnjZ2B9LGXMUigwue25kZnNfMXCWgdcFgj8JKQjOEcB-ERE1GNBdJWnXTCH5e9zq-bOb817IDSoEhFjtp6trvnQyKfPf3CAG33T' style='height: 200px; width: 200px;'/>

In [128]:
# Imputaciones generales
df_copy_q0 = df.copy()

df_copy_q0['answer_q0'] = generic_data_cleansing(df_copy_q0['answer_q0'])\
    .map(lambda x: x.replace(', ', '/'))\
    .map(lambda x: x.replace(' o ', '/'))\
    .map(lambda x: x.replace(' / ', '/'))

get_top_3_answers_by_country(df_copy_q0, 'answer_q0')

country,answer_q0,count
Argentina,Reposera,189
Argentina,Silleta,5
Argentina,Silla/reposera,3
Bolivia,Silla,1
Brasil,Cadeira de praia,1
Chile,Silla de playa,27
Chile,Reposera,4
Chile,Silla,2
Colombia,Silla de playa,3
Colombia,Silla,2


#### Pregunta: ¿Cómo se llama esta prenda en tu país? (answer_q1)
Imagen: <img src='https://lh4.googleusercontent.com/kVkb9k2so2VZwx4wR8lA8BmC6DULtzDUkL3owk9FkLb-Oe71v7pOmDFMl0F3r4Pt-azlQbzeIUTRNL9PaPKR1HgtkewuWWiuvq4d6ZZo9xHzmdsbyi3oi9_4GGSqh1BdYbcobcKFl6Arz03J' style='height: 200px; width: 200px;'/>

In [129]:
df_copy_q1 = df.copy()

df_copy_q1['answer_q1'] = generic_data_cleansing(df_copy_q1['answer_q1'])\
    .map(lambda x: x.replace('(bata)', 'Bata'))\
    .map(lambda x: x.replace('Tunica', 'Túnica'))\
    .map(lambda x: x.replace('El.guardapolvo', 'Guardapolvo'))

get_top_3_answers_by_country(df_copy_q1, 'answer_q1')

country,answer_q1,count
Argentina,Guardapolvo,153
Argentina,Delantal,21
Argentina,Guardapolvos,10
Bolivia,Mandil,1
Brasil,Avental,1
Chile,Delantal,21
Chile,Cotona,6
Chile,Bata,4
Colombia,Bata,12
Colombia,Delantal,2


#### Pregunta: ¿Cómo llaman a este objeto en tu país? (answer_q2)
Imagen: <img src='https://lh5.googleusercontent.com/bAWVUc2WylKgt-UbpP7d9y8n46_RRe7x5YeZ3YSQBFEOYQjoNrYDC8gMM2KHYcMmlgBAR0CL-xF74_mpjW6ATF11-riOZK0INfbWNOXOwcWPyjdPe_stifLrXkapAjhZro-C8vUmj62q7gXd' style='height: 200px; width: 200px;'/>

In [130]:
df_copy_q2 = df.copy()

df_copy_q2['answer_q2'] = generic_data_cleansing(df_copy_q2['answer_q2'])\
    .map(lambda x: x.split('.')[0])

get_top_3_answers_by_country(df_copy_q2, 'answer_q2')

country,answer_q2,count
Argentina,Pulsera,137
Argentina,Pulseras,55
Argentina,Pulceras,7
Bolivia,Manilla,1
Brasil,Pulseiras,1
Chile,Pulsera,27
Chile,Pulseras,15
Chile,Pulcera,1
Colombia,Pulsera,7
Colombia,Pulseras,5


#### Pregunta: ¿Cómo se llama este objeto en tu país? (answer_q3)
Imagen: <img src='https://lh6.googleusercontent.com/aZRw5A-5Ktao4COYc905lxChLPZmR7Chv5SGdkngvSBzCPwhi2XypYxGsYc7hKdFv-dBnDkdqxij82vuqv1yQ9wcGBq1C-ac9IY7lwUYHGagJ8LJCkvyu5lm2rIxFQOFoa5sO6aNpMqWmEyM' style='height: 200px; width: 200px;'/>

In [131]:
df_copy_q3 = df.copy()

df_copy_q3['answer_q3'] = generic_data_cleansing(df_copy_q3['answer_q3'])

get_top_3_answers_by_country(df_copy_q3, 'answer_q3')

country,answer_q3,count
Argentina,Anillo,204
Argentina,Pulsera,2
Argentina,Alianza,1
Bolivia,Anillo,1
Brasil,Bracelete,1
Chile,Anillo,38
Chile,Anillo de compromiso,1
Chile,"Anillo de compromiso, ilusion.",1
Colombia,Anillo,13
Colombia,Anillo argolla,2


#### Pregunta: ¿Cómo se le dice a esta prenda en tu país? (answer_q4)
Imagen: <img src='https://lh3.googleusercontent.com/Uci-sRJ2aJKSx1t3ATU3HI78JR6SAHfav5xuIr0JSS4x4ckDBry6VqKdwiOmYRsoMipp7Tt1iIynfxMXgG5Bxo3KIp0A7Od9_4cxrUcMqFtQZwS5okHYQWjTx2Ad7tKmBTobWQE-m8mrGoLv' style='height: 200px; width: 200px;'/>

In [132]:
df_copy_q4 = df.copy()

df_copy_q4['answer_q4'] = generic_data_cleansing(df_copy_q4['answer_q4'])

get_top_3_answers_by_country(df_copy_q4, 'answer_q4')

country,answer_q4,count
Argentina,Remera,203
Argentina,Remera manga corta,2
Argentina,(remera) mangas cortas,1
Bolivia,Polera,1
Brasil,Camiseta,1
Chile,Polera,39
Chile,Camiseta,1
Chile,Polera manga corta a rayas,1
Colombia,Camiseta,11
Colombia,Camisa,2


#### Pregunta: ¿Qué son estos? (answer_q5)
Imagen: <img src='https://lh5.googleusercontent.com/98wBW1nc7XisFr0c_wAq9X-7kbqtR5lyzEbkdDTB9pgOyOjD5u1vDlXtpXZJuzKPHl9fyrF5OG2AD5Ah3nK6uO1LdiRW5aojBCj-CtW0FkDrDzuQTbw4pY1qKFK0oJ9HxBwc_TPfkUW1cQTf' style='height: 200px; width: 200px;'/>

In [133]:
df_copy_q5 = df.copy()

df_copy_q5['answer_q5'] = generic_data_cleansing(df_copy_q5['answer_q5'])

get_top_3_answers_by_country(df_copy_q5, 'answer_q5')

country,answer_q5,count
Argentina,Diarios,171
Argentina,Diario,34
Argentina,Diarios o periódicos,2
Bolivia,Periódico,1
Brasil,Jornais,1
Chile,Diarios,29
Chile,Diario,10
Chile,Diario/periodico,1
Colombia,Periódicos,11
Colombia,Periódico,3


#### Pregunta: ¿Cómo le llaman a esto en tu país? (answer_q6)
Imagen: <img src='https://lh5.googleusercontent.com/p8oZJ7egqWojPscOcYDGeuMSpbNzjnqiY8UBpXZawuIUnHZei3TCRLbBcjCNDSezyeX9g11SkEfZFRvlX-P0c_3LYVfjlx62bm0bJCrF6EfH1N5lo9gVsHxlP4ujrr6Xwn96N39OKEEopGG6' style='height: 200px; width: 200px;'/>

In [134]:
df_copy_q6 = df.copy()

df_copy_q6['answer_q6'] = generic_data_cleansing(df_copy_q6['answer_q6'])

get_top_3_answers_by_country(df_copy_q6, 'answer_q6')

country,answer_q6,count
Argentina,Cava,89
Argentina,Heladera,23
Argentina,Vinoteca,17
Bolivia,Nevera sorvete,1
Brasil,Adega,1
Chile,Frigobar,9
Chile,Cava,6
Chile,Minibar,3
Colombia,Nevera,7
Colombia,Nevera para vinos,2


#### Pregunta: ¿Cómo se le dice a este objeto en su país? (answer_q7)
Imagen: <img src='https://lh4.googleusercontent.com/YRpC-a8nwoV3zjI96qfpxcKjnHCQDIufLA66e3a7ZoJSYaAnfMNmZp32szszvci--GUmJMY-OA31QF_UWKknwIVMGsFJJh081I1BtR9WtSB_60Ne4oSx7R3MxmivtSgyJUKUeYvoD9P00IV_' style='height: 200px; width: 200px;'/>

In [135]:
df_copy_q7 = df.copy()

df_copy_q7['answer_q7'] = generic_data_cleansing(df_copy_q7['answer_q7'])

get_top_3_answers_by_country(df_copy_q7, 'answer_q7')

country,answer_q7,count
Argentina,Sorbete,51
Argentina,Pajita,45
Argentina,Pajita o sorbete,14
Bolivia,Sorbete,1
Brasil,Canudos,1
Chile,Bombilla,24
Chile,Bombillas,5
Chile,Pajita,5
Colombia,Pitillo,12
Colombia,Silicona,2


#### Pregunta: ¿Cómo se le dice a este objeto en tu país? (answer_q8)
Imagen: <img src='https://lh5.googleusercontent.com/GJsVHR4GT4OA7M0jkopZcHQaisXf6HbeBdQ4IiwsUOjGQwc1Ar6RXRDnCMjr5M2gBL7arj8H4yAFGa6aytSEbQJtm2i5Zo7DrE_yrJO0pJvVyZSBeFyO1IGZ7N7HFJ07gRr7WIik0JxZUPgP' style='height: 200px; width: 200px;'/>

In [136]:
df_copy_q8 = df.copy()

df_copy_q8['answer_q8'] = generic_data_cleansing(df_copy_q8['answer_q8'])

get_top_3_answers_by_country(df_copy_q8, 'answer_q8')

country,answer_q8,count
Argentina,Moño,212
Argentina,Lazo,1
Argentina,Moñocinta,1
Bolivia,Moña,1
Brasil,Laço,1
Chile,Cinta,10
Chile,Moño,9
Chile,Cinta de regalo,3
Colombia,Moño,14
Colombia,Lazo/moño,1


#### Pregunta: ¿Cómo se le dice a este objeto en tu país? (answer_q9)
Imagen: <img src='https://lh4.googleusercontent.com/19vxMzIrVYUnGUGcZ5wPfOj8aE0uhysw06PY-dAIv8jFosowlLATrIcNaFgzCGDhLRkjkcovVoLcFBCu6GPNtlciu-ccBDuLf695wK-QlJ1nJhaxFaK_cfeVUvFGdb3xav2JWiyhh4WnkI38' style='height: 200px; width: 200px;'/>

In [137]:
df_copy_q9 = df.copy()

df_copy_q9['answer_q9'] = generic_data_cleansing(df_copy_q9['answer_q9'])

get_top_3_answers_by_country(df_copy_q9, 'answer_q9')

country,answer_q9,count
Argentina,Cinta scotch,83
Argentina,Cinta,59
Argentina,Cinta adhesiva,23
Bolivia,Diurex,1
Brasil,Durex,1
Chile,Scotch,30
Chile,Scotch o cinta adhesiva,3
Chile,Cinta,1
Colombia,Cinta,10
Colombia,Cinta pegante,4


#### Pregunta: ¿Cómo llaman a esto en tu país? (answer_q10)
Imagen: <img src='https://lh4.googleusercontent.com/XTIqsu9YPSEMQUSQ86pvWQGFDBcT3ZVjqCPfxVJUzjyxjRzX5vRg7O3sffi49B68p0X6mBRlpeatCtdVKBBIVuVXyI0-5AW7hM1Wn7bnCYpIUWr2VPXkEN-G4Y26g4ad1Zp37XCOMeVbrfoS' style='height: 200px; width: 200px;'/>

In [138]:
df_copy_q10 = df.copy()

df_copy_q10['answer_q10'] = generic_data_cleansing(df_copy_q10['answer_q10'])

get_top_3_answers_by_country(df_copy_q10, 'answer_q10')

country,answer_q10,count
Argentina,Resaltador,132
Argentina,Resaltadores,54
Argentina,Marcador,5
Bolivia,Resaltador,1
Brasil,Marca texto,1
Chile,Destacador,35
Chile,Destacadores,5
Chile,"Marcador, destacador.",1
Colombia,Resaltador,10
Colombia,Resaltadores,4


#### Pregunta: ¿Cómo le dicen a estas/os en tu país? (answer_q11)
Imagen: <img src='https://lh5.googleusercontent.com/RuQkgHuXPWmLBppLVs_o_3N1Ffd6_8Txvn6D3Y-PDekQ5NaH6PBPpdu9nbVJhaP0HkX22DMnz9vJ9JPHLSzb_1RVLCOjYwSIexmRDxbHsMsYzimRBaCOB-_k1NbwLl4nRcib4u7sU6fMehGQ' style='height: 200px; width: 200px;'/>

In [139]:
df_copy_q11 = df.copy()

df_copy_q11['answer_q11'] = generic_data_cleansing(df_copy_q11['answer_q11'])

get_top_3_answers_by_country(df_copy_q11, 'answer_q12')

country,answer_q12,count
Argentina,Vincha,140
Argentina,vincha,35
Argentina,Vincha,17
Bolivia,Bincha,1
Brasil,Tiara,1
Chile,Cintillo,26
Chile,cintillo,7
Chile,Cintillo,5
Colombia,Balaca,3
Colombia,Diadema,2


#### Pregunta: ¿Cómo llaman a este accesorio en tu país? (answer_q12)
Imagen: <img src='https://lh5.googleusercontent.com/2Tfh-fXZyjpJjYqc_DAJyWbnhplx1dQAHNxg7U8JDpThwink0G3tQYe4ztg7qXrftSyEYhmv9FeraChsgSFItYaGl3zfmnsoH_RAukNiKnNR8vYuOZohySMaTbZOq0sBqMiE_bdV8VUwesrK' style='height: 200px; width: 200px;'/>

In [140]:
df_copy_q12 = df.copy()

df_copy_q12['answer_q12'] = generic_data_cleansing(df_copy_q12['answer_q12'])

get_top_3_answers_by_country(df_copy_q12, 'answer_q12')

country,answer_q12,count
Argentina,Vincha,198
Argentina,Bincha,10
Argentina,No sé,1
Bolivia,Bincha,1
Brasil,Tiara,1
Chile,Cintillo,41
Chile,Cintillo de plástico,1
Chile,Cintillo.,1
Colombia,Balaca,5
Colombia,Diadema,4


#### Pregunta: ¿Cómo se le dice a esto en tu país? (answer_q13)
Imagen: <img src='https://lh3.googleusercontent.com/aM9QCGkuAh60d_SM-NrSqq2TpdEZj-EvfeIMIN5iIXYSo1io7-VhOA7Q8nZ4MAkTefb-9o2Sj3LCt6qR_H_8cGRCJH_0eR66VkiYmo-81PO0wwcaTKRaiqF8YdLj4gWOtEddOF7Y9BvJ66eY' style='height: 200px; width: 200px;'/>

In [141]:
df_copy_q13 = df.copy()

df_copy_q13['answer_q13'] = generic_data_cleansing(df_copy_q13['answer_q13'])

get_top_3_answers_by_country(df_copy_q13, 'answer_q13')

country,answer_q13,count
Argentina,Pochoclera,90
Argentina,Pochoclo,65
Argentina,Pochoclos,17
Bolivia,Pipokas,1
Brasil,Carrinho de pipoca,1
Chile,Cabritas,20
Chile,Palomitas,7
Chile,Palomitas de maíz,3
Colombia,Crispetas,3
Colombia,Maiz pira,2


#### Pregunta: ¿Cómo llaman a este elemento en tu país? (answer_q14)
Imagen: <img src='https://lh3.googleusercontent.com/e-2441zawnwl6SDKnfSfC9E3s1CVo0rPU86rbsbbBHTatYFIbw013id5HQCyXGdYMq0qd0tyFv3gEUitK5WuOZZuoAdd77N47bHpobWd-lulHFSNtDeglZQteJQ5pAEC-cUBDgj5BHLDtAAj' style='height: 200px; width: 200px;'/>

In [142]:
df_copy_q14 = df.copy()

df_copy_q14['answer_q14'] = generic_data_cleansing(df_copy_q14['answer_q14'])

get_top_3_answers_by_country(df_copy_q14, 'answer_q14')

country,answer_q14,count
Argentina,Bolsa de dormir,204
Argentina,Carpa,4
Argentina,Saco de dormir,2
Bolivia,No se ve la imagen,1
Brasil,Saco de proteção,1
Chile,Saco de dormir,42
Chile,Saco de dormir.,1
Colombia,Sleeping,5
Colombia,Bolsa para dormir,3
Colombia,Sleeping bag,2


#### Pregunta: ¿Cómo le dicen a este utensilio en tu país? (answer_q15)
Imagen: <img src='https://lh3.googleusercontent.com/aWO3_LzBkbaNEsPAl0htWWWRkppRDT9fQoO4o-W63xz2iv0tDhTo1bS8cDk08PPqDqyxSyKruBqyyYBotebnasA5jU_UTET88WRKIu4qp2scGs2RuKIvcLbOc8t5uLJ-B8xSXwMDhE53fJcL' style='height: 200px; width: 200px;'/>

In [143]:
df_copy_q15 = df.copy()

df_copy_q15['answer_q15'] = generic_data_cleansing(df_copy_q15['answer_q15'])\
    .map(lambda x: x.replace('Espatula', 'Espátula'))

get_top_3_answers_by_country(df_copy_q15, 'answer_q15')

country,answer_q15,count
Argentina,Espátula,184
Argentina,Espátula de silicona,5
Argentina,Espátula de goma,3
Bolivia,Lengua,1
Brasil,Espátula,1
Chile,Espátula,15
Chile,Mezquino,9
Chile,Paleta,4
Colombia,Espátula,11
Colombia,-,1


#### Pregunta: ¿Cómo llaman a este elemento en tu país? (answer_q16)
Imagen: <img src='https://lh5.googleusercontent.com/ISE9MiAPdoptT_yHMHWAuh-V4tnyFvPbxFxU8rTXwctZIWs_-dmmfvGUQzOI0QdJX50P58yEX2oEUWd7n7vGibQXoQJl8fPd_nFT5Ffr4AxqfFU6UqjjkxpfjsZEx3z0NR9ntFvPZk8e4mt2' style='height: 200px; width: 200px;'/>

In [144]:
df_copy_q16 = df.copy()

df_copy_q16['answer_q16'] = generic_data_cleansing(df_copy_q16['answer_q16'])

get_top_3_answers_by_country(df_copy_q16, 'answer_q16')

country,answer_q16,count
Argentina,Inflable,53
Argentina,Inflables,37
Argentina,Flotador,36
Bolivia,Flotador,1
Brasil,Boia para piscina,1
Chile,Flotador,28
Chile,Flotadores,10
Chile,Inflable,3
Colombia,Flotador,9
Colombia,Flotadores,6


#### Pregunta: ¿Cómo llaman a esto en tu país? (answer_q17)
Imagen: <img src='https://lh5.googleusercontent.com/_jVpGcWCgNvdIwHFqDqcrn6Y7bt9_TGXc_EdeZRUnjPHiUEpCYn8v4lmbgnifu1DeHjPxgKLLKsQPtow58FGaAKViHwzvaUgXHyVJPXGS2fX3vW-v9lcgD10_AGJULDTNsSlMaTXI4Ohq_YK' style='height: 200px; width: 200px;'/>

In [145]:
df_copy_q17 = df.copy()

df_copy_q17['answer_q17'] = generic_data_cleansing(df_copy_q17['answer_q17'])

get_top_3_answers_by_country(df_copy_q17, 'answer_q17')

country,answer_q17,count
Argentina,Cuaderno,119
Argentina,Agenda,23
Argentina,Libreta,17
Bolivia,Agenda,1
Brasil,Caderno,1
Chile,Agenda,10
Chile,Cuaderno,10
Chile,Libreta,8
Colombia,Agenda,5
Colombia,Libreta,4


#### Pregunta: ¿Cómo llaman a esto en tu país? (answer_q18)
Imagen: <img src='https://lh4.googleusercontent.com/AQHSzN4JX3z73GQ0nyAp1yhZXsy-DKV2I6lhHWxyCSrL1hAvLv6XLhg9Tk2q5ZNPJr6fNeLfjKi4MVK9mlmwoFRYSMYTxN-BO6noXg-UmcbVYTppxKJTWbzlSj_jdI2oFIcY9QJvawnUWcZe' style='height: 200px; width: 200px;'/>

In [146]:
df_copy_q18 = df.copy()

df_copy_q18['answer_q18'] = generic_data_cleansing(df_copy_q18['answer_q18'])

get_top_3_answers_by_country(df_copy_q18, 'answer_q18')

country,answer_q18,count
Argentina,Lapicera,113
Argentina,Birome,51
Argentina,Lapicera o birome,8
Bolivia,Lapicera,1
Brasil,Caneta,1
Chile,Lápiz pasta,17
Chile,Lápiz,5
Chile,Lapiz pasta,4
Colombia,Esfero,3
Colombia,Bolígrafo,2


#### Pregunta: ¿Cómo se le dice a esto en tu país? (answer_q19)
Imagen: <img src='https://lh4.googleusercontent.com/XvEvDGFkCSWYhiNCljEwUpMewJFCEy8RBlIFeH-vVMMFVh2rdw4gRwxNtN87yuo9dbvTxodic_Qyr8gExqWA3TbVvmRqkXMRcJW2E8cXFLmxv7ex6prVEn5Us_HOFy7GDoXZqIlNKf1I0erM' style='height: 200px; width: 200px;'/>

In [147]:
df_copy_q19 = df.copy()

df_copy_q19['answer_q19'] = generic_data_cleansing(df_copy_q19['answer_q19'])

get_top_3_answers_by_country(df_copy_q19, 'answer_q19')

country,answer_q19,count
Argentina,Pelotero,210
Argentina,,1
Argentina,No recuerdo,1
Bolivia,Pelotero,1
Brasil,Piscina de bolinha,1
Chile,Piscina de pelotas,26
Chile,Pelotas,4
Chile,,1
Colombia,Piscina de pelotas,13
Colombia,Pelotas,2


#### Pregunta: ¿Cómo llaman a este juego en tu país? (answer_q20)
Imagen: <img src='https://lh4.googleusercontent.com/TbWxCLrBKe0Hbm9TettqxNQk1z694ZrQYUKsxoA-XEkOpyCrGgGjX6XIhNYhS33VVchHNXVBUIzP563y4PwBxqYc5gCncxvttXMeNjFXhNhEHh5aoDEm-xZ2jdnaF3iJaU7V1lZOSi8fqugv' style='height: 200px; width: 200px;'/>

In [148]:
df_copy_q20 = df.copy()

df_copy_q20['answer_q20'] = generic_data_cleansing(df_copy_q20['answer_q20'])

get_top_3_answers_by_country(df_copy_q20, 'answer_q20')

country,answer_q20,count
Argentina,Metegol,203
Argentina,Mete gol,3
Argentina,Fulbito,2
Bolivia,Futbolin,1
Brasil,Pimbolim,1
Chile,Taca taca,31
Chile,Taca-taca,8
Chile,Tacataca,2
Colombia,Futbolito,8
Colombia,Futbolín,3


#### Pregunta: ¿Cómo se le dice a este juego en tu país? (answer_q21)
Imagen: <img src='https://lh3.googleusercontent.com/VNICTkvPHFMCSIn3ErMCbyUqKyeC3U2xylkRdYRza0ewXtKRhDP1-9BBT7bGLEo-IO77en9HeMNthEgjkp0q5XAkSdzbAoEzs2jxZRYZV90H893DppSuzcSxv65kaboM0RYuNta57ighn5Kk' style='height: 200px; width: 200px;'/>

In [149]:
df_copy_q21 = df.copy()

df_copy_q21['answer_q21'] = generic_data_cleansing(df_copy_q21['answer_q21'])

get_top_3_answers_by_country(df_copy_q21, 'answer_q21')

country,answer_q21,count
Argentina,Calesita,178
Argentina,Calecita,19
Argentina,Carrusel,9
Bolivia,Calesita,1
Brasil,Carrossel,1
Chile,Carrusel,38
Chile,Caballitos,1
Chile,Carrousel,1
Colombia,Carrusel,16
Costa rica,Carrucel,1


#### Pregunta: ¿Cómo se le dice a esto en tu país? (answer_q22)
Imagen: <img src='https://lh4.googleusercontent.com/xGMEZdGESML-UeqHC6Yy_8pjK5cGZocvplaWTcZYZ8nxNr0zsYPQY1GaCB_kZ6XcYYNARj0CeVZTH4O2Sp55rm1M_QxYkK56laz-p76Cr3I-a4mRXg9oxnY9T1Vkwi-1t3cE4Onu37BE4Bcw' style='height: 200px; width: 200px;'/>

In [150]:
df_copy_q22 = df.copy()

df_copy_q22['answer_q22'] = generic_data_cleansing(df_copy_q22['answer_q22'])

get_top_3_answers_by_country(df_copy_q22, 'answer_q22')

country,answer_q22,count
Argentina,Hamaca,153
Argentina,Hamacas,50
Argentina,Amaca,4
Bolivia,Columpio,1
Brasil,Gangorra,1
Chile,Columpio,30
Chile,Columpios,12
Chile,Columpios.,1
Colombia,Columpio,9
Colombia,Culumpio,3


#### Pregunta: ¿Cómo se le dice a esto en tu país? (answer_q23)
Imagen: <img src='https://lh5.googleusercontent.com/b7jhTSoe1yXY_1QMsi9Benn5Soh5WT99WKJRwuwfK8oQlQj54-Wj_rmuoeweSNTG5T1VVQjzH1PESuNe63_bxZ6GYLRrErgFoSS0ZbxExN_oNxLaMvK3tRL5NQXIK-xi5j-jvA2QE6rMQt9G' style='height: 200px; width: 200px;'/>

In [151]:
df_copy_q23 = df.copy()

df_copy_q23['answer_q23'] = generic_data_cleansing(df_copy_q23['answer_q23'])

get_top_3_answers_by_country(df_copy_q23, 'answer_q23')

country,answer_q23,count
Argentina,Tobogán,165
Argentina,Tobogan,48
Argentina,Resbalada,1
Bolivia,Resbalador,1
Brasil,Escorregador,1
Chile,Resbalín,21
Chile,Resfalin,6
Chile,Resbalin,4
Colombia,Rodadero,8
Colombia,Tobogán,3


#### Pregunta: ¿Cómo se le dice al lugar donde se compran estas cosas en tu país? (answer_q24)
Imagen: <img src='https://lh4.googleusercontent.com/-RPUo5X4icUJiOhVmWrxWZun_vhC1i0wSX6mxgdvbVXWJTnoq0DNDGJs-sS_MDINTWbsS50Ms-FvnY2J_w4EmOx8j0bdqVy2ZOCvQpBdXOMx9x02G8aZkr7aqysVji-CjwdeeibDJWFcYRM6' style='height: 200px; width: 200px;'/>

In [152]:
df_copy_q24 = df.copy()

df_copy_q24['answer_q24'] = generic_data_cleansing(df_copy_q24['answer_q24'])\
    .map(lambda x: x.replace('Libreria', 'Librería'))

get_top_3_answers_by_country(df_copy_q24, 'answer_q24')

country,answer_q24,count
Argentina,Librería,192
Argentina,Librerías,3
Argentina,Góndola,2
Bolivia,Librería,1
Brasil,Papelaria,1
Chile,Librería,29
Chile,,1
Chile,Artículos de papelería,1
Colombia,Papelería,10
Colombia,Marcadores,1


#### Pregunta: ¿Cómo se le dice a esto en tu país? (answer_q25)
Imagen: <img src='https://lh6.googleusercontent.com/9uzYDMBa5gzvHxpMiAJlwKl8fmX85ZmNIxnSlVbUgB59TlPNr0JsXlgzMdoCW9O7J5_JTTlLF_Lni9MNCRR4IvHlyMP1IR_KIpkfmdKmL95JQ7LelB9ubPNxCHHA3hEphmzkmRjoKa53_tyv' style='height: 200px; width: 200px;'/>

In [153]:
df_copy_q25 = df.copy()

df_copy_q25['answer_q25'] = generic_data_cleansing(df_copy_q25['answer_q25'])

get_top_3_answers_by_country(df_copy_q25, 'answer_q25')

country,answer_q25,count
Argentina,Sube y baja,113
Argentina,Subibaja,74
Argentina,Subeybaja,3
Bolivia,Subibaja,1
Brasil,Balanço,1
Chile,Sube y baja,28
Chile,Balancín,7
Chile,Balancin,3
Colombia,Sube y baja,10
Colombia,,1


#### Pregunta: ¿Cómo llaman a esta fruta en tu país? (answer_q26)
Imagen: <img src='https://lh3.googleusercontent.com/X0U6d65mcbj1P812GstZ6GkSM0oWDd7qzIUtLSbu6uwr7oX8mKAjOS2qCFQ0p3SNVbZBNaUxnJgMmE0byTWLoAzoZxFFNMb9lGBCKTYgMhx9W1wLQr7Lmjt2SxbYbHgkFOv7LySyFesZtBD2' style='height: 200px; width: 200px;'/>

In [154]:
df_copy_q26 = df.copy()

df_copy_q26['answer_q26'] = generic_data_cleansing(df_copy_q26['answer_q26'])

get_top_3_answers_by_country(df_copy_q26, 'answer_q26')

country,answer_q26,count
Argentina,Frutilla,212
Argentina,Frutillas,1
Argentina,Frutrilla,1
Bolivia,Frutilla,1
Brasil,Morango,1
Chile,Frutilla,41
Chile,"Frutilla, kelleñ",1
Chile,Frutilla.,1
Colombia,Fresa,16
Costa rica,Fresa,1


#### Pregunta: ¿Cómo llaman a este dulce en tu país? (answer_q27)
Imagen: <img src='https://lh4.googleusercontent.com/ASpe63-YkVwlET86HVWWgIzIzKe9NX4E5oEWNz5LCIDwaz8JhnFYiIQ_CAaiJKAZeYlY-rpcWvR8_DNSMBU1Aw-iyaJVEQy1rFWu6dRG-8mYAaiy5fLHGe7Qv1RLbDD7fYm3XHWzheGpZ6Ey' style='height: 200px; width: 200px;'/>

In [155]:
df_copy_q27 = df.copy()

df_copy_q27['answer_q27'] = generic_data_cleansing(df_copy_q27['answer_q27'])

get_top_3_answers_by_country(df_copy_q27, 'answer_q27')

country,answer_q27,count
Argentina,Dulce de leche,208
Argentina,De leche,1
Argentina,Dulce de lche,1
Bolivia,Dulce de leche,1
Brasil,Brigadeiro,1
Chile,Manjar,42
Chile,Manjar blanco,1
Colombia,Arequipe,15
Colombia,Arequipe / manjar blanco,1
Costa rica,Dulce de leche,1


#### Pregunta: ¿Cómo llaman a este alimento en tu país? (answer_q28)
Imagen: <img src='https://lh5.googleusercontent.com/sJYWU5vYH9dqLpkxHPgDNkGLJdS-St7vJI51K7591E5JgWPleloDlVUqjC3DmHtbIaU-zVYkTxrGoND3jW1QXXcOkpMJI9ftDVi45byDXoXxRqRm_ChRwGVh1o4CF0eXZD2uDL65hSd_LAZd' style='height: 200px; width: 200px;'/>

In [156]:
df_copy_q28 = df.copy()

df_copy_q28['answer_q28'] = generic_data_cleansing(df_copy_q28['answer_q28'])\
    .map(lambda x: x.replace('Mani', 'Maní'))

get_top_3_answers_by_country(df_copy_q28, 'answer_q28')

country,answer_q28,count
Argentina,Maní,201
Argentina,Maníes,7
Argentina,Maní salado,2
Bolivia,Maní,1
Brasil,Amendoim,1
Chile,Maní,42
Chile,Maní salado,1
Colombia,Maní,14
Colombia,Maní salado,1
Colombia,Maíz,1


#### Pregunta: ¿Cómo llaman a esta bebida en tu país? (answer_q29)
Imagen: <img src='https://lh3.googleusercontent.com/zb3usOWBAGVbiE9Lt_ZqAxeXjyoTSZDg0gb2lyNUT-Gh2qjOCQQhYNsTRb3fn_tkVod6NVNdKPmTPikWrMYa_Vx4_aSSl_QdI-0fhRDW94lAHK4sVnkj7Rmj6kZ_a1_Kv2ebilDE4snnyLdy' style='height: 200px; width: 200px;'/>

In [157]:
df_copy_q29 = df.copy()

df_copy_q29['answer_q29'] = generic_data_cleansing(df_copy_q29['answer_q29'])

get_top_3_answers_by_country(df_copy_q29, 'answer_q29')

country,answer_q29,count
Argentina,Cerveza,160
Argentina,Cerveza/birra,14
Argentina,Birra,9
Bolivia,cerveza,1
Brasil,Cerveja- chopp,1
Chile,Cerveza,34
Chile,"Cerveza, chela",2
Chile,Cervesa,1
Colombia,Cerveza,8
Colombia,Cerveza o pola,2


#### Pregunta: ¿Cómo le llaman a esta verdura en tu país? (answer_q30)
Imagen: <img src='https://lh3.googleusercontent.com/tvYrBjXM1IRs-RFWtDbtnIhyn7cGFE3qh31luXAG0zR8J8VcIVOcjU-DqOLQHOxScPY0DTKCGwjAehMWc9rnPINXAo7kY4H1dGasKel_fyWZTiM57v8hQi-eYEzYsPLE7kayN4WMRGSnerQR' style='height: 200px; width: 200px;'/>

In [158]:
df_copy_q30 = df.copy()

df_copy_q30['answer_q30'] = generic_data_cleansing(df_copy_q30['answer_q30'])

get_top_3_answers_by_country(df_copy_q30, 'answer_q30')

country,answer_q30,count
Argentina,Remolacha,186
Argentina,Rabanito,7
Argentina,Cebolla,5
Bolivia,Remolacha,1
Brasil,Beterraba,1
Chile,Betarraga,26
Chile,Beterraga,11
Chile,Rábano,2
Colombia,Remolacha,13
Colombia,,1


#### Pregunta: ¿Cómo llaman a este snack en tu país? (answer_q31)
Imagen: <img src='https://lh4.googleusercontent.com/tEAoRO9NYH_tnpx1U9OJMcbO_YRQYkcX7zjgiAQx9w806-eRoayR_TfegKt1Ena5Ih3vjlhOI3jDyNY4XYAkaZ3mJCqWSGrp4KxaQwW3HxSw9UiklxAApNa4OpQXcGudpK4yXb8MhxbqY0rN' style='height: 200px; width: 200px;'/>

In [159]:
df_copy_q31 = df.copy()

df_copy_q31['answer_q31'] = generic_data_cleansing(df_copy_q31['answer_q31'])

get_top_3_answers_by_country(df_copy_q31, 'answer_q31')

country,answer_q31,count
Argentina,Pochoclo,133
Argentina,Pochoclos,37
Argentina,Pororo,6
Bolivia,Pipokas,1
Brasil,Pipoca,1
Chile,Cabritas,20
Chile,Palomitas,7
Chile,Cabrita,3
Colombia,Crispetas,6
Colombia,Maíz pira,3


#### Pregunta: ¿Como llaman a esta bebida en tu país? (answer_q32)
Imagen: <img src='https://lh5.googleusercontent.com/VD6tQEU5R5ZMh9n2udaI-9I3O0sOooMlPSMFijZfUmAdc84FMBaA7DCcd_2JM5Ko8hMzTAcJQ6dK8t1pFDq7L6zzDg9S1Uj8bmH9nMBO0i6LyBZJThYWtSSMr1yGSi7JhMEv01eODoQtFp-u' style='height: 200px; width: 200px;'/>

In [160]:
df_copy_q32 = df.copy()

df_copy_q32['answer_q32'] = generic_data_cleansing(df_copy_q32['answer_q32'])

get_top_3_answers_by_country(df_copy_q32, 'answer_q32')

country,answer_q32,count
Argentina,Gaseosa,75
Argentina,Coca cola,38
Argentina,Coca,36
Bolivia,Soda,1
Brasil,Não reconheço,1
Chile,Bebida,15
Chile,Coca cola,10
Chile,Coca-cola,3
Colombia,Gaseosa,12
Colombia,Coca cola?,1


#### Pregunta: ¿Cómo llaman a este snack en tu país? (answer_q33)
Imagen: <img src='https://lh5.googleusercontent.com/-_vWQTUPSN-RR405sVQaIkytkew8hq3RBt_-r9KJyBCNMx5cnDE1RQ_37v4go01mDlFmQl3RZDOk-NpF51mFqnAMQaxKT2c3-TDm1ZsRhgfTJ92XrHuXE3EzHS-V-mMEnuyi26aXb_W3iG0k' style='height: 200px; width: 200px;'/>

In [161]:
df_copy_q33 = df.copy()

df_copy_q33['answer_q33'] = generic_data_cleansing(df_copy_q33['answer_q33'])

get_top_3_answers_by_country(df_copy_q33, 'answer_q33')

country,answer_q33,count
Argentina,Papas fritas,149
Argentina,Papitas,27
Argentina,Papa frita,5
Bolivia,Papas fritas,1
Brasil,Salgadinho,1
Chile,Papas fritas,35
Chile,Papitas fritas,2
Chile,Papa frita,1
Colombia,Papas fritas,4
Colombia,Papitas,4


#### Pregunta: ¿Cómo llaman a esta verdura en tu país? (answer_q34)
Imagen: <img src='https://lh4.googleusercontent.com/-4xp3Jc659_MV3SvY8B6Ab_DCTcCBM8k7iJT-RDnh4iyBPCK7u4AxxD7sjOxNVYYeb5iq4DilqXz5ORVifbC2Em1rwpUOT4VMqMxO7W_XJG4kYe5GlFnGE5f3qjvKYoB94-w1RoPoFUwmzmz' style='height: 200px; width: 200px;'/>

In [162]:
df_copy_q34 = df.copy()

df_copy_q34['answer_q34'] = generic_data_cleansing(df_copy_q34['answer_q34'])

get_top_3_answers_by_country(df_copy_q34, 'answer_q34')

country,answer_q34,count
Argentina,Choclo,204
Argentina,Maíz,3
Argentina,Choclo/maíz,2
Bolivia,choclo,1
Brasil,. milho,1
Chile,Choclo,41
Chile,Choclo.,1
Chile,Maíz/choclo,1
Colombia,Mazorca,13
Colombia,Choclo,2


#### Pregunta: ¿Cómo llaman a este dulce en tu país? (answer_q35)
Imagen: <img src='https://lh3.googleusercontent.com/1J851l8A7XVSwq6GyLR9A8JLIOys7Ilce55CWtfDAQeXJceo9H31bYkscBqn4gqEPrhnj3r_ADa8Lo5yjk8Sw0jLDbOVYJgRmi9xFEitx1_oPy73IObUBs5evf1DTP9mmWFKPWOHIi0GPX_y' style='height: 200px; width: 200px;'/>

In [163]:
df_copy_q35 = df.copy()

df_copy_q35['answer_q35'] = generic_data_cleansing(df_copy_q35['answer_q35'])\
    .map(lambda x: x.replace('Chupetin', 'Chupetín'))

get_top_3_answers_by_country(df_copy_q35, 'answer_q35')

country,answer_q35,count
Argentina,Chupetín,206
Argentina,Chuletín,2
Argentina,Chupetín bolita,2
Bolivia,chupetei,1
Brasil,Pirulito,1
Chile,Coyac,15
Chile,Koyak,5
Chile,Chupete,3
Colombia,Bombón,3
Colombia,Bom bom bum,2


#### Pregunta: ¿Cómo llaman a esta fruta en tu país? (answer_q36)
Imagen: <img src='https://lh5.googleusercontent.com/qIx2Gssv12VYBENclsd5iVIMJTHicwT-NxGcqakHdFC0PNjXtYEGtlPvTLAblWdwS0Rv_aLP-4RUQPDdu3SC-AdezBlF2mXeX7YUAd6_u4H-uki6m9zBovRqgMCECgWFzfB5v41a0G-kg3JD' style='height: 200px; width: 200px;'/>

In [164]:
df_copy_q36 = df.copy()

df_copy_q36['answer_q36'] = generic_data_cleansing(df_copy_q36['answer_q36'])\
    .map(lambda x: x.replace('Anana', 'Ananá'))

get_top_3_answers_by_country(df_copy_q36, 'answer_q36')

country,answer_q36,count
Argentina,Ananá,192
Argentina,Piña,7
Argentina,Ananá/piña,3
Bolivia,Piñs,1
Brasil,Abacaxi,1
Chile,Piña,43
Colombia,Piña,16
Costa rica,Piña,1
Cuba,Piña,1
Ecuador,Piña,8


#### Pregunta: ¿Cómo llaman a esta verdura en tu país? (answer_q37)
Imagen: <img src='https://lh4.googleusercontent.com/vWO5FqY6BotX-hM-WVhKDamCwAoMC4olK1w8Pcsbw9sLWZs96wonYQxy1LGV_DgIKJfarOoXpbZnfk7B5J5yk4gLQ93Dew9tCn2W166hsAOHUcfrhsUnRE8QgAyj4S0Bi_Uj91BxGekq82UL' style='height: 200px; width: 200px;'/>

In [165]:
df_copy_q37 = df.copy()

df_copy_q37['answer_q37'] = generic_data_cleansing(df_copy_q37['answer_q37'])

get_top_3_answers_by_country(df_copy_q37, 'answer_q37')

country,answer_q37,count
Argentina,Calabaza,80
Argentina,Zapallo,48
Argentina,Zapallo anco,30
Bolivia,Anco,1
Brasil,Abóbora,1
Chile,Zapallo,23
Chile,Calabaza,5
Chile,Zapallo butternut,4
Colombia,Ahuyama,3
Colombia,Calabaza,2


#### Pregunta: ¿Cómo llaman a esta verdura en tu país? (answer_q38)
Imagen: <img src='https://lh6.googleusercontent.com/s9gt9R0OYQxUua2CUTlFxhh1AqgaQB7V9kiKbsAmgzX5-sOWYLF-nRYhuIYt-6ToDqwBcVpiD1s-gsbmiWn3dEjhQr0qS6DWUomVa6UmuijPm5kIiEm2EHfbaym5QkC8j0LSBDz_Gk6Zyof5' style='height: 200px; width: 200px;'/>

In [166]:
df_copy_q38 = df.copy()

df_copy_q38['answer_q38'] = generic_data_cleansing(df_copy_q38['answer_q38'])

get_top_3_answers_by_country(df_copy_q38, 'answer_q38')

country,answer_q38,count
Argentina,Zapallo,105
Argentina,Calabaza,58
Argentina,Zapallo cabutia,11
Bolivia,Zapallo,1
Brasil,Abóbora,1
Chile,Zapallo,35
Chile,Zapallo camote,4
Chile,Calabaza,2
Colombia,Auyama,5
Colombia,Ahuyama,4


#### Pregunta: ¿Cómo llaman a este producto en tu país? (answer_q39)
Imagen: <img src='https://lh3.googleusercontent.com/4RgWgmtJzJ3s_tY6IBnPWl_TrJykaabKFAX6DSzvhTc6WIP7m67w8x_QVynkW_V2HtfxR0mAL7KaLkFtsQEYBju7qy0JfhNcdHnpZrNi0FIph7k-ifbJ8pHeGyAzOAyCAskHEIv0zMPARq71' style='height: 200px; width: 200px;'/>

In [167]:
df_copy_q39 = df.copy()

df_copy_q39['answer_q39'] = generic_data_cleansing(df_copy_q39['answer_q39'])

get_top_3_answers_by_country(df_copy_q39, 'answer_q39')

country,answer_q39,count
Argentina,Leche condensada,175
Argentina,Crema,10
Argentina,Crema de leche,5
Bolivia,Leche condensada,1
Brasil,Leite condensado,1
Chile,Leche condensada,43
Colombia,Leche condensada,10
Colombia,Lecherita,3
Colombia,Leche condenzada,1
Costa rica,Leche evaporada,1


#### Pregunta: ¿Cómo llaman a este dulce en tu país? (answer_q40)
Imagen: <img src='https://lh6.googleusercontent.com/tUhs5wHyQ44a9WtLTz07HVvd-SeOt3VeWwh3hYmLEJBi9c4UCT_XnhL1R58OAcYmSUAV1JSB29cG15I6xohhRy8JZ5VYXQeQSETzS9U8RuJRpnjf3SpXybn7c-i_Gtwrk9oiSxCigdsB4Zok' style='height: 200px; width: 200px;'/>

In [168]:
df_copy_q40 = df.copy()

df_copy_q40['answer_q40'] = generic_data_cleansing(df_copy_q40['answer_q40'])

get_top_3_answers_by_country(df_copy_q40, 'answer_q40')

country,answer_q40,count
Argentina,Chicle,203
Argentina,Bazuca,1
Argentina,Chicke,1
Bolivia,Chicle,1
Brasil,Chiclete,1
Chile,Chicle,41
Chile,Chicle globito,1
Chile,Chicle.,1
Colombia,Chicle,15
Colombia,Bomba de chicle,1


#### Pregunta: ¿Cómo llaman a este tipo de azúcar en tu país? (answer_q41)
Imagen: <img src='https://lh3.googleusercontent.com/-0ia5AE5u_GYLO1LbWp_KO5g30qd_LaZ5-RQ08aJ38TpI4xHWDYa09ehikX8MpKV6sr82JzCsgWBAp1V5QTlTK6awjFgBxIjSeGZ7_nz6xTXdjS5QSVVZJqxjVBIBMcSiskLAcwV3WrN1K4w' style='height: 200px; width: 200px;'/>

In [169]:
df_copy_q41 = df.copy()

df_copy_q41['answer_q41'] = generic_data_cleansing(df_copy_q41['answer_q41'])

get_top_3_answers_by_country(df_copy_q41, 'answer_q41')

country,answer_q41,count
Argentina,Azúcar impalpable,93
Argentina,Impalpable,80
Argentina,Azucar impalpable,29
Bolivia,Azúcar impalpable,1
Brasil,Refinado,1
Chile,Azúcar flor,32
Chile,Azucar flor,9
Chile,Azúcar flor.,1
Colombia,Azúcar en polvo,3
Colombia,Azúcar pulverizada,2


#### Pregunta: ¿Cómo le llaman a este estado en tu país? (Malestar por haber tomado demasiado) (answer_q42)
Imagen: <img src='https://lh3.googleusercontent.com/VdgF7FlsCKT3CAxV4tL5Bv-_WYzjJ81uZJHkJLJ6S4P9DaiwZiRUUdqNxHQiIvzI5DUA2OHPiWja409cFCPWfbEvwed3eayPcqGe4eo4TMqFik7Izs6EsWIlZSJC2t9eEp5SV5MB-s_zgnzd' style='height: 200px; width: 200px;'/>

In [170]:
df_copy_q42 = df.copy()

df_copy_q42['answer_q42'] = generic_data_cleansing(df_copy_q42['answer_q42'])

get_top_3_answers_by_country(df_copy_q42, 'answer_q42')

country,answer_q42,count
Argentina,Resaca,201
Argentina,Borracho,3
Argentina,Borrachera,1
Bolivia,Chaki,1
Brasil,Enxaqueca,1
Chile,Caña,20
Chile,Estar con la caña,3
Chile,Resaca,3
Colombia,Guayabo,15
Colombia,Resaca,1


#### Pregunta: ¿Cómo le dicen a este tipo de persona en tu país? (Persona con mucho dinero/ de una zona de mucho dinero que suele comportarse algo arrogante o creerse superior por tener dinero) (answer_q43)
Imagen: <img src='https://lh5.googleusercontent.com/3bEDtVs1LfWoc4ouH1MtEtThSe-fNTTpyjcriAQ9xA9zEY00zfdbAFsLIycEZU-RhpbQdT9X6XLZ6xXVbc13YeCpUvD7bxdKxUGjtkGp1BeX-u3MZ6ZSUxyKgK1HBIqYjDU04xBK8kuy5Wa0' style='height: 200px; width: 200px;'/>

In [171]:
df_copy_q43 = df.copy()
df_copy_q43['answer_q43'] = generic_data_cleansing(df_copy_q43['answer_q43'])

get_top_3_answers_by_country(df_copy_q43, 'answer_q43')

country,answer_q43,count
Argentina,Cheto,56
Argentina,Cheta,52
Argentina,Cheto/a,17
Bolivia,Fresa,1
Brasil,Metida a besta,1
Chile,Cuico,19
Chile,Cuica,3
Chile,Cuica/cuico,3
Colombia,Gomelo,2
Colombia,Gomelos,2
